In [1]:
import geopandas as gpd
import os
from glob import glob
import pandas as pd
import datetime
pd.set_option('display.max_columns', 500)
print('start')
deaths = gpd.read_file("../migrant_deaths_analysis/death_files/ogis_migrant_deaths_091021_with_yuma.geojson")
deaths=deaths[deaths['Latitude'].notnull()]
deaths = deaths.set_crs('epsg:4326')
print(1)
deaths['Latitude']= deaths.Latitude.astype(float)
deaths['Longitude']= deaths.Longitude.astype(float)
deaths = gpd.GeoDataFrame(deaths, geometry=gpd.points_from_xy(deaths.Longitude,deaths.Latitude))

print(2)

deaths['Date']= pd.to_datetime(deaths['Reporting Date'])
deaths['year'] = deaths['Date'].dt.year
deaths['month'] = deaths['Date'].dt.month

print(3)

#shapefile created in qgis from files in ALL_F477_2020_06_Mobile_Voice_Deployment-by_Provider
gdf= gpd.GeoDataFrame.from_file('../migrant_deaths_analysis/deaths_in_cell_service/voice_2020_6/voice_2020_6.shp')
gdf
gdf = gdf.to_crs('epsg:4326')
print(4)

deaths

deaths.columns

start
1
2
3
4


Index(['ML Number', 'Name', 'Sex', 'Age', 'Reporting Date',
       'Surface Management', 'Location', 'Location Precision', 'Corridor Code',
       'Corridor', 'Cause of Death', 'OME Determined COD', 'Condition Code',
       'Body Condition', 'Post Mortem Interval', 'State', 'County', 'Latitude',
       'Longitude', 'UTM X', 'UTM Y', 'Date', 'OME number', 'YCSO number',
       'Nationality', 'merge_id', 'geometry', 'year', 'month'],
      dtype='object')

In [21]:
# deaths[deaths['ML Number'].isna()]

In [22]:
deaths_with_coverage = gpd.sjoin(deaths, gdf, how="left", op='intersects')

deaths_with_coverage



deaths_with_coverage_unique = deaths_with_coverage.drop_duplicates(subset=deaths.columns)


/Users/adevoid/opt/anaconda3/envs/pyqgis-dev/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  """Entry point for launching an IPython kernel.


In [23]:
deaths_with_coverage_unique['DBA'].value_counts(dropna=False)


VerizonWireless    1573
T-Mobile            797
Sprint              585
NaN                 514
AT&T Mobility       443
Name: DBA, dtype: int64

In [24]:
deaths_with_coverage_unique[deaths_with_coverage_unique['year']==2020]['DBA'].value_counts(dropna=False)

VerizonWireless    100
NaN                 66
AT&T Mobility       26
T-Mobile            24
Sprint              23
Name: DBA, dtype: int64

In [25]:
deaths_with_coverage_unique[deaths_with_coverage_unique['year']==2021]['DBA'].value_counts(dropna=False)

VerizonWireless    88
NaN                48
AT&T Mobility      33
T-Mobile           29
Sprint             23
Name: DBA, dtype: int64

In [26]:
deaths_with_coverage_unique[(deaths_with_coverage_unique['month']==6)&(deaths_with_coverage_unique['year']==2021)]['DBA'].value_counts(dropna=False)



VerizonWireless    19
Sprint              9
AT&T Mobility       9
NaN                 8
T-Mobile            7
Name: DBA, dtype: int64

In [27]:
deaths_with_coverage_unique[(deaths_with_coverage_unique['DBA'].isnull())&(deaths_with_coverage_unique['year']==2021)]



,ML Number,Name,Sex,Age,Reporting Date,Surface Management,Location,Location Precision,Corridor Code,Corridor,Cause of Death,OME Determined COD,Condition Code,Body Condition,Post Mortem Interval,State,County,Latitude,Longitude,UTM X,UTM Y,Date,OME number,YCSO number,Nationality,merge_id,geometry,year,month,index_right,id,DBA,Technology,TECHNOLO_1
3274,21-0312,Unidentified,male,,1/11/21,Tohono Oodham Nation,,GPS Coordinate (precise to within ca. 300ft./1...,8,Cowlick,Skeletal Remains,UNDETERMINED (SKELETAL REMAINS),7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,PIMA,31.833750,-112.374300,369952,3522831,2021-01-11,None,None,None,21-0312,POINT (-112.37430 31.83375),2021.0,1.0,NaN,NaN,NaN,NaN,NaN
3282,21-0635,Unidentified,male,,1/25/21,Cabeza Prieta National Wildlife Refuge,,GPS Coordinate (precise to within ca. 300ft./1...,11,Goldwater,Skeletal Remains,Undetermined-Skeletal Remains,7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,PIMA,32.131000,-113.237833,288903,3557149,2021-01-25,None,None,None,21-0635,POINT (-113.23783 32.13100),2021.0,1.0,NaN,NaN,NaN,NaN,NaN
3283,21-0653,Unidentified,undetermined,,1/28/21,Tohono Oodham Nation,,GPS Coordinate (precise to within ca. 300ft./1...,7,San Miguel,Skeletal Remains,UNDETERMINED - SKELETAL REMAINS,7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,PIMA,31.714160,-111.957910,409240,3509152,2021-01-28,None,None,None,21-0653,POINT (-111.95791 31.71416),2021.0,1.0,NaN,NaN,NaN,NaN,NaN
3287,21-0803,Unidentified,male,,2/7/21,US Forest Service,,GPS Coordinate (precise to within ca. 300ft./1...,6,Sasabe,Skeletal Remains,Undetermined-skeletal remains,7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,SANTA CRUZ,31.389361,-111.230389,478097,3472777,2021-02-07,None,None,None,21-0803,POINT (-111.23039 31.38936),2021.0,2.0,NaN,NaN,NaN,NaN,NaN
3294,21-0996,Unidentified,undetermined,,2/21/21,Cabeza Prieta National Wildlife Refuge,,GPS Coordinate (precise to within ca. 300ft./1...,10,Cabeza Prieta,Skeletal Remains,Undetermined - skeletal remains,7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,PIMA,32.225240,-113.049530,306869,3567244,2021-02-21,None,None,None,21-0996,POINT (-113.04953 32.22524),2021.0,2.0,NaN,NaN,NaN,NaN,NaN
3295,21-0997,Unidentified,male,,2/21/21,Cabeza Prieta National Wildlife Refuge,,GPS Coordinate (precise to within ca. 300ft./1...,10,Cabeza Prieta,Skeletal Remains,Undetermined - skeletal remains,7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,PIMA,32.225360,-113.049410,306880,3567257,2021-02-21,None,None,None,21-0997,POINT (-113.04941 32.22536),2021.0,2.0,NaN,NaN,NaN,NaN,NaN
3298,21-1046,Unidentified,male,,2/24/21,Cabeza Prieta National Wildlife Refuge,,GPS Coordinate (precise to within ca. 300ft./1...,10,Cabeza Prieta,Skeletal Remains,Undetermined - skeletal remains,7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,PIMA,32.119500,-113.171800,295108,3555746,2021-02-24,None,None,None,21-1046,POINT (-113.17180 32.11950),2021.0,2.0,NaN,NaN,NaN,NaN,NaN
3299,21-1132,Unidentified,male,,3/3/21,Organ Pipe Cactus National Monument,,GPS Coordinate (precise to within ca. 300ft./1...,10,Cabeza Prieta,Skeletal Remains,undetermined — skeletal remains,7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,PIMA,32.199638,-113.021598,309448,3564356,2021-03-03,None,None,None,21-1132,POINT (-113.02160 32.19964),2021.0,3.0,NaN,NaN,NaN,NaN,NaN
3302,21-1270,Unidentified,male,,3/14/21,Organ Pipe Cactus National Monument,,GPS Coordinate (precise to within ca. 300ft./1...,10,Cabeza Prieta,Skeletal Remains,undetermined -- skeletal remains,6,Complete skeletonization w/ disarticulation,< 6-8 months,Arizona,PIMA,32.092970,-112.962560,314798,3552426,2021-03-14,None,None,None,21-1270,POINT (-112.96256 32.09297),2021.0,3.0,NaN,NaN,NaN,NaN,NaN
3304,21-1281,Unidentified,undetermined,,3/13/21,Organ Pipe Cactus National Monument,,GPS Coordinate (precise to within ca. 300ft./1

In [28]:
def tec(s):
    
    if pd.isnull(s['TECHNOLO_1']):
    
        return s['Technology']
    else:
        return s['TECHNOLO_1']    
    

gdf['tec'] = gdf.apply(lambda row : tec(row), axis = 1)

def nam(s):
    return str(s['tec'])+'_'+s['DBA']
     

gdf['name'] = gdf.apply(lambda row : nam(row), axis = 1)
gdf['name'].value_counts(dropna=False)

85.0_Sprint                       819
83.0_T-Mobile                     364
86.0_T-Mobile                     304
85.0_VerizonWireless               44
83.0_AT&T Mobility                 42
86.0_AT&T Mobility                 40
86.0_CellularOne of NE Arizona      1
85.0_CellularOne of NE Arizona      1
Name: name, dtype: int64

In [29]:



gdf1 = gdf.groupby('tec')['DBA'].value_counts(dropna=False).to_frame()
gdf1=gdf1.rename(columns={"DBA": "DBA_count1"})
gdf1 = gdf1.reset_index()

# gdf1


def nam(s):
    return str(s['tec'])+'_'+s['DBA']
     

gdf1['name'] = gdf1.apply(lambda row : nam(row), axis = 1)
    






In [30]:
import numpy as np


gdf['geometry']= gdf.buffer(0)
# deaths_2020_6['geometry'] = deaths_2020_6.buffer(0)

unique_carrers = gdf1['name'].to_list()

print(unique_carrers)

tec = gdf1['tec'].to_list()
DBA =gdf1['DBA'].to_list()

def funx(x,ca):
    
    carrier_tec= gpd.GeoDataFrame(gdf[gdf['name']==ca], crs="EPSG:4326")
#     print(carrier_tec['geometry'])
#     print(x['geometry'])
#     print(len(carrier_tec))
#     print(np.sum(carrier_tec.intersects(x['geometry'])))
    return np.sum(carrier_tec.intersects(x['geometry']))

for en,ca in enumerate(unique_carrers):
    print(ca)
    deaths[ca] = deaths.apply(lambda x:funx(x,ca),axis=1)
    




['83.0_T-Mobile', '83.0_AT&T Mobility', '85.0_Sprint', '85.0_VerizonWireless', '85.0_CellularOne of NE Arizona', '86.0_T-Mobile', '86.0_AT&T Mobility', '86.0_CellularOne of NE Arizona']
83.0_T-Mobile
83.0_AT&T Mobility
85.0_Sprint
85.0_VerizonWireless
85.0_CellularOne of NE Arizona
86.0_T-Mobile
86.0_AT&T Mobility
86.0_CellularOne of NE Arizona


In [31]:
deaths['coverage total']= deaths['83.0_T-Mobile']+deaths['83.0_AT&T Mobility']+deaths['85.0_Sprint']+deaths['85.0_VerizonWireless']+deaths['85.0_CellularOne of NE Arizona']+deaths['86.0_T-Mobile']+deaths['86.0_AT&T Mobility']+deaths['86.0_CellularOne of NE Arizona']



In [32]:
deaths

,ML Number,Name,Sex,Age,Reporting Date,Surface Management,Location,Location Precision,Corridor Code,Corridor,Cause of Death,OME Determined COD,Condition Code,Body Condition,Post Mortem Interval,State,County,Latitude,Longitude,UTM X,UTM Y,Date,OME number,YCSO number,Nationality,merge_id,geometry,year,month,83.0_T-Mobile,83.0_AT&T Mobility,85.0_Sprint,85.0_VerizonWireless,85.0_CellularOne of NE Arizona,86.0_T-Mobile,86.0_AT&T Mobility,86.0_CellularOne of NE Arizona,coverage total
0,00-00097,Unidentified,male,,1/10/00,Cocopah Indian Reservation,,"physical description with directions, distance...",12,Yuma,Skeletal Remains,UNDETERMINED - MUMMIFIED AND SKELETAL REMAINS,4,Skeletonization w/ mummification,< 5 weeks,Arizona,YUMA,32.589668,-114.799807,143294,3612175,2000-01-10,None,None,None,00-00097,POINT (-114.79981 32.58967),2000.0,1.0,1,1,1,2,0,1,1,0,7
1,00-00148,"Nicolas, Tomas",male,17,1/22/00,State of Arizona,,"physical description with directions, distance...",4,Patagonia,Exposure,HYPOTHERMIA,1,Fully fleshed,< 1 day,Arizona,PIMA,31.812406,-110.483205,548914,3519758,2000-01-22,None,None,None,00-00148,POINT (-110.48320 31.81241),2000.0,1.0,0,1,0,2,0,0,1,0,4
2,00-00189,Unidentified,male,,1/20/00,US Forest Service,,"physical description with directions, distance...",5,Nogales,Skeletal Remains,UNDETERMINED - SKELETAL REMAINS,7,Complete skeletonization w/ bone degradation,> 6-8 months,Arizona,SANTA CRUZ,31.568000,-111.151300,485643,3492563,2000-01-20,None,None,None,00-00189,POINT (-111.15130 31.56800),2000.0,1.0,1,1,0,2,0,0,1,0,5
3,00-00222,Unidentified,male,,1/10/00,Private,,"physical description with directions, distance...",9,Ajo,Skeletal Remains,UNDETERMINED - SKELETAL REMAINS,6,Complete skeletonization w/ disarticulation,< 6-8 months,Arizona,PIMA,32.355997,-112.826855,328102,3581363,2000-01-10,None,None,None,00-00222,POINT (-112.82685 32.35600),2000.0,1.0,1,0,0,2,0,1,0,0,4
4,00-00245,"Candia-Bravo, Maria Del Rocio",female,20,2/5/00,Tohono Oodham Nation,,"physical description with directions, distance...",7,San Miguel,Blunt Force Injury,MULTIPLE BLUNT FORCE INJURIES,1,Fully fleshed,< 1 day,Arizona,PIMA,31.717790,-111.779830,426116,3509419,2000-02-05,None,None,None,00-00245,POINT (-111.77983 31.71779),2000.0,2.0,0,0,0,2,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3909,None,None,Male,22,8/31/21,Cabeza Prieta,None,None,None,None,None,UNDETERMINED - SKELETAL REMAINS,None,None,None,None,None,32.363920,-113.401930,None,None,2021-08-31,2021-1141,2021-70152,Guatemala,2021-1141,POINT (-113.40193 32.36392),2021.0,8.0,0,0,0,0,0,0,0,0,0
3910,None,None,Female,37,9/10/21,Yuma,None,None,None,None,None,Exposure to High Outdoor Temperature,None,None,None,None,None,32.460636,-114.616562,None,None,2021-09-10,2021-1182,2021-73187,Mexican,2021-1182,POINT (-114.61656 32.46064),2021.0,9.0,1,1,1,2,0,0,1,0,6
3911,None,None,Female,42,9/10/21,Yuma,None,None,None,None,None,Exposure to High Outdoor Temperatures,None,None,None,None,None,32.472349,-114.608894,None,None,2021-09-10,2021-1183,2021-73259,Mexican,2021-1183,POINT (-114.60889 32.47235),2021.0,9.0,1,1,1,2,0,1,1,0,7
3912,None,None,Male,33,9/10/21,Yuma,None,None,None,None,None,Exposure to High Outdoor Temperatures,None,None,None,None,None,32.478630,-114.617300,None,None,2021-09-10,2021-73360,2021-73360,Mexican,2021-73360,POINT (-114.61730 32.47863),2021.0,9.0,1,1,1,2,0,1,1,0,7


In [33]:
deaths.to_csv('../migrant_deaths_analysis/deaths_in_cell_service/deaths_cell_coverage.csv')

In [47]:
print(len(deaths))

3912


In [34]:
# deaths = gpd.GeoDataFrame.from_file("../migrant_deaths/deaths/deaths.geojson")
# print(1)
# bounding_box = deaths.envelope
# print(2)
# frame1 = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])


# frame1.to_file("death_bounds.shp")

In [35]:
deaths['Date']= pd.to_datetime(deaths['Reporting Date'])
# result['year'] = result['Date'].dt.year
# result['month'] = result['Date'].dt.month
deaths_6_20 = deaths[(deaths['Date']>= pd.Timestamp('2020-06-01'))]

deaths['coverage total'].value_counts(dropna=False)

7    973
3    762
2    673
0    514
4    420
5    294
6    225
1     51
Name: coverage total, dtype: int64

In [36]:
gdf.to_file('cell_2020_buffer.shp')

In [37]:
deaths_6_20['coverage total'].value_counts(dropna=False)

2    100
0     84
7     78
3     59
5     18
4     17
6     15
1      5
Name: coverage total, dtype: int64

In [38]:
print(len(deaths))

3912


In [46]:
deaths[(deaths['Date']>= pd.Timestamp('2000-01-01'))]['coverage total'].value_counts(dropna=False)

7    894
3    742
2    669
0    510
4    400
5    292
6    210
1     51
Name: coverage total, dtype: int64

In [40]:
deaths[(deaths['Date']>= pd.Timestamp('2021-01-01'))]['coverage total'].value_counts(dropna=False)

2    58
0    48
7    47
3    35
4    12
5    10
6     9
1     2
Name: coverage total, dtype: int64

In [45]:
deaths[(deaths['Date']< pd.Timestamp('2021-01-01'))&(deaths['Date']>= pd.Timestamp('2020-01-01'))]['coverage total'].value_counts(dropna=False)

0    66
2    51
3    45
7    41
4    13
5    11
6     9
1     3
Name: coverage total, dtype: int64

In [44]:
deaths[(deaths['Date']>= pd.Timestamp('2021-06-01'))&(deaths['Date']<= pd.Timestamp('2021-06-30'))]['coverage total'].value_counts(dropna=False)

7    14
3    12
2    10
0     8
4     4
5     3
6     1
Name: coverage total, dtype: int64

In [43]:
deaths[(deaths['Date']>= pd.Timestamp('2021-06-01'))&(deaths['Date']<= pd.Timestamp('2021-06-30'))]

,ML Number,Name,Sex,Age,Reporting Date,Surface Management,Location,Location Precision,Corridor Code,Corridor,Cause of Death,OME Determined COD,Condition Code,Body Condition,Post Mortem Interval,State,County,Latitude,Longitude,UTM X,UTM Y,Date,OME number,YCSO number,Nationality,merge_id,geometry,year,month,83.0_T-Mobile,83.0_AT&T Mobility,85.0_Sprint,85.0_VerizonWireless,85.0_CellularOne of NE Arizona,86.0_T-Mobile,86.0_AT&T Mobility,86.0_CellularOne of NE Arizona,coverage total
3347,21-2200,"Ramirez Carranza, Filiberto",male,40,6/1/21,Private,,Street Address (precise to within ca. 1000ft/3...,5,Nogales,Exposure,Probable hyperthermia,1,Fully fleshed,< 1 day,Arizona,SANTA CRUZ,31.417830,-110.962654,503551,3475910,2021-06-01,None,None,None,21-2200,POINT (-110.96265 31.41783),2021.0,6.0,1,1,1,2,0,1,1,0,7
3348,21-2222,Unidentified,male,,6/3/21,Tohono Oodham Nation,,GPS Coordinate (precise to within ca. 300ft./1...,7,San Miguel,Exposure,Probable hyperthermia,1,Fully fleshed,< 1 day,Arizona,PIMA,32.008629,-111.747142,429435,3541635,2021-06-03,None,None,None,21-2222,POINT (-111.74714 32.00863),2021.0,6.0,1,0,0,2,0,0,0,0,3
3349,21-2230,"Chocojay Umul, Byron",male,20,6/3/21,Tohono Oodham Nation,,"physical description with directions, distance...",7,San Miguel,Blunt Force Injury,blunt force injuries,1,Fully fleshed,< 1 day,Arizona,PIMA,31.962486,-111.750348,429097,3536523,2021-06-03,None,None,None,21-2230,POINT (-111.75035 31.96249),2021.0,6.0,1,0,0,2,0,0,0,0,3
3350,21-2252,"Jimenez Godinez, Juan",male,20,6/5/21,Tohono Oodham Nation,,GPS Coordinate (precise to within ca. 300ft./1...,7,San Miguel,Exposure,Probable hyperthermia,2,Decomposed,< 1 week,Arizona,PIMA,32.270086,-111.850464,419905,3570690,2021-06-05,None,None,None,21-2252,POINT (-111.85046 32.27009),2021.0,6.0,1,0,0,2,0,0,0,0,3
3351,21-2259,Unidentified,male,,6/5/21,Private,,GPS Coordinate (precise to within ca. 300ft./1...,1,Bernadino,Exposure,PROBABLE HYPERTHERMIA,2,Decomposed,< 1 week,Arizona,COCHISE,31.494300,-109.217000,669345,3485761,2021-06-05,None,None,None,21-2259,POINT (-109.21700 31.49430),2021.0,6.0,0,0,0,2,0,0,0,0,2
3352,21-2267,"Castro Jaime, Marvin",male,47,6/6/21,Private,,Street Address (precise to within ca. 1000ft/3...,5,Nogales,Exposure,PROBABLE HYPERTHERMIA,1,Fully fleshed,< 1 day,Arizona,PIMA,31.589732,-111.239393,477289,3494986,2021-06-06,None,None,None,21-2267,POINT (-111.23939 31.58973),2021.0,6.0,1,1,1,2,0,1,1,0,7
3353,21-2283,"Gomez Lopez, Beatriz",female,27,6/7/21,US Forest Service,,GPS Coordinate (precise to within ca. 300ft./1...,5,Nogales,Blunt Force Injury,"blunt force injury, head",1,Fully fleshed,< 1 day,Arizona,PIMA,31.542690,-111.234460,477746,3489771,2021-06-07,None,None,None,21-2283,POINT (-111.23446 31.54269),2021.0,6.0,1,1,1,2,0,1,1,0,7
3354,21-2293,"Reyes Martinez, Leobardo",male,45,6/8/21,Private,,GPS Coordinate (precise to within ca. 300ft./1...,5,Nogales,Exposure,Probable hyperthermia,1,Fully fleshed,< 1 day,Arizona,PIMA,31.739910,-111.005163,499512,3511607,2021-06-08,None,None,None,21-2293,POINT (-111.00516 31.73991),2021.0,6.0,1,1,1,2,0,1,1,0,7
3355,21-2294,"Ruiz Toala, Gilberto",male,24,6/8/21,Private,,GPS Coordinate (precise to within ca. 300ft./1...,7,San Miguel,Exposure,Probable hyperthermia,2,Decomposed,< 1 week,Arizona,PINAL,32.816206,-112.001540,406246,3631355,2021-06-08,None,None,None,21-2294,POINT (-112.00154 32.81621),2021.0,6.0,1,1,1,2,0,1,1,0,7
3356,21-2318,Unidentified,female,,6/11/21,Organ Pipe Cactus National Monument,,GPS Coordinate (precise to within ca. 300ft./1...,10,Cabeza Prieta,Skeletal Remains,Undetermined-Skeletal and Mummified Remains,5,Skeletonization w/ articulation/ligamentous at...,< 3 months,Arizona,PIMA,31.946783,-112.883883,321941,3536086,2021-06-11,None,None,None,21-2318,POINT (-112.88388 31.94678),2021.0,6.0,0,0,0,0,0,0,0,0,0


In [50]:
pd.set_option('display.max_rows', 615)
deaths_6_20.groupby(['year','month'])['coverage total'].value_counts(dropna=False).to_frame()

coverage total
year   month coverage total                
2020.0 6.0   2                            7
             0                            3
             3                            3
             7                            1
       7.0   7                           11
             0                            7
             2                            6
             3                            4
             1                            1
             4                            1
             5                            1
       8.0   3                            8
             2                            7
             0                            5
             5                            4
             4                            2
             6                            1
       9.0   7                           11
             2                           10
             0                            6
             3                            5
             1                            1
             4                            1
             5                            1
             6                            1
       10.0  2                            6
             6                            4
             7                            4
             0                            2
             3                            2
             5                            1
       11.0  0                            5
             2                            4
             3                            2
             7                            2
             5                            1
       12.0  0                            8
             2                            2
             7                            2
             1                            1
             4                            1
2021.0 1.0   2                            6
             0                            3
             7                            3
             3                            2
             4                            2
             5                            2
       2.0   2                            6
             0                            5
             3                            2
             4                            1
             5                            1
       3.0   0                            8
             2                            6
             7                            3
             4                            1
             6                            1
       4.0   2                            4
             0                            3
             3                            3
             7                            2
             4                            1
             5                            1
             6                            1
       5.0   7                            8
             0                            6
             2                            3
             3                            3
             4                            1
             5                            1
             6                            1
       6.0   7                           14
             3                           12
             2                           10
             0                            8
             4                            4
             5                            3
             6                            1
       7.0   2                            9
             3                            4
             0                            3
             7                            2
             1                            1
             4                            1
             5                            1
             6                            1
       8.0   7                           10
             0                            7
             2                            4
             3           

In [53]:




deaths_6_20.to_file('../migrant_deaths_analysis/deaths_in_cell_service/deaths_6_2020.geojson',driver='GeoJSON')

In [11]:



deaths = pd.read_csv('../migrant_deaths_analysis/deaths_in_cell_service/deaths_cell_coverage.csv')
deaths['Date']= pd.to_datetime(deaths['Reporting Date'])
deaths_6_20 = deaths[(deaths['Date']>= pd.Timestamp('2020-06-01'))]

deaths_6_20['coverage total'].value_counts(dropna=False).to_frame()



,coverage total
2,100
0,84
7,78
3,59
5,18
4,17
6,15
1,5


In [12]:
deaths['coverage total'].value_counts(dropna=False).to_frame()

,coverage total
7,973
3,762
2,673
0,514
4,420
5,294
6,225
1,51
